<a href="https://colab.research.google.com/github/sualeh/introduction-to-chatgpt-api/blob/colab/chatgpt-api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

----------

> **How to Run This Notebook**

To get started, create an Open AI API account, set up billing, and generate and API key at https://platform.openai.com/. If you are running the notebook locally in Visual Studio Code or other IDE, create a file called `.env`, and add a line `OPENAI_API_KEY=<your-openai-api-key>`. This key will be read by the `load_dotenv` library.

Otherwise, if you are running in Google Colab, create a secret called `OPENAI_API_KEY` and set it to the value of your OpenAI API key.

Run the code below to read the key.


In [ ]:
%pip install python-dotenv

from os import environ as env
from dotenv import load_dotenv

# Load key from an environmental variable called "OPENAI_API_KEY"
# Use python-dotenv https://pypi.org/project/python-dotenv/
# And take environment variables from .env
load_dotenv()
try:
  # Attempt to read OPENAI_API_KEY from a Google Colab secret
  from google.colab import userdata
  env['OPENAI_API_KEY'] = env.get('OPENAI_API_KEY', userdata.get('OPENAI_API_KEY'))
except ModuleNotFoundError:
  print("Not running in Google Colab")
  # No action - rely on the OPENAI_API_KEY environmental variable



----------

# A Short Introduction to the ChatGPT API



The ChatGPT is relatively easy to use. It has object bindings in most major languages, so you can use your language of choice.


## Getting Started

In [ ]:
%pip install openai

from openai import OpenAI

# Create Open AI client to use ChatGPT
client = OpenAI()

## Making a Request

Start writing your first request - you need to specify the model and provide the user prompt.

The "user" role represents the user's input or message within the conversation. The prompt can include questions, commands, or any other content the user wants to communicate.

In [ ]:
prompt = "When was the first moon landing?"

chat_completion = client.chat.completions.create(
    messages=[
        {"role": "user", "content": prompt}
    ],
    model="gpt-3.5-turbo",
)

print(chat_completion)

The completion has the response content, as well as usage information. Responses are messages from the "agent" role. What we really want to extract, though, is the actual message content.

In [ ]:
reply = chat_completion.choices[0].message.content
print(reply)

## Using the "system" Role

The "system" role is used to provide high-level instructions or setting the behavior of the assistant. It guides the conversation or specifies the desired outcome. This information is typically provided at the beginning of the conversation.

In [ ]:
prompt = "When was the first moon landing?"

chat_completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "Always reply in French."},
        {"role": "user", "content": prompt},
    ],
)

reply = chat_completion.choices[0].message.content
print(reply)

ChatGPT is most useful when it has context from the conversation. You will need to maintain chat history yourself, by keeping the "system" and "user" messages as well as the "assistant" reponses in a list, and passing this list into every API call.

## Maintaining Context

ChatGPT is a REST API, and does not maintain sessions. However, the client can maintain it's own chat history, and present that on an API call for additional context on the conversation. To be effective, you would need to maintain both the user questions and the ChatGPT responses. In it's simplest form, it is a list of chat messages.

In [ ]:
prompt = "What is the moon's circumference in km?"

message_history = []
message_history.append(
    {"role": "user", "content": prompt}
)

chat_completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=message_history
)

# Print the response:
reply_content = chat_completion.choices[0].message.content
print(reply_content)

Let us store the response in the history as well, so we can make a context-sensitive prompt.

In [ ]:
message_history.append({"role": "assistant", "content": f"{reply_content}"})

Now, ask an ambiguous question, which is answered in context.

In [ ]:
prompt = "How far is it?"

message_history.append({"role": "user", "content": prompt})

chat_completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=message_history
)

reply_content = chat_completion.choices[0].message.content
print(reply_content)

## References

- Based on [Sentdex/ChatGPT-API-Basics](https://github.com/Sentdex/ChatGPT-API-Basics)
- See [ChatGPT API in Python by sentdex](https://www.youtube.com/watch?v=c-g6epk3fFE)